In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt

from pandas_datareader.data import DataReader

In [ ]:
from matplotlib import rcParams

# Restore old behavior of rounding default axis ranges
rcParams['axes.autolimit_mode'] = 'round_numbers'
rcParams['axes.xmargin'] = 0
rcParams['axes.ymargin'] = 0

In [ ]:
dta_pl = pd.read_stata('data/data_PL.dta')
dta_pl.head()

In [ ]:
yy = (2000+(dta_pl['t']-1)//4).astype(str)
qq = (1+(dta_pl['t']-1)%4).astype(str)
date = yy+'Q'+qq

from statsmodels.tsa.base.datetools import dates_from_str
date = dates_from_str(date)

dta_pl.index = date

dta_pl = dta_pl.resample('QS').mean()

dta_pl.tail()

In [ ]:
model_pl = sm.tsa.VAR(dta_pl[['GDP','HICP','WIB3M']])

In [ ]:
print(model_pl.select_order(8))
model_pl.select_order(8).summary()

In [ ]:
results_pl = model_pl.fit(6)

In [ ]:
irf_pl = results_pl.irf(20)
irf_pl.plot(orth=True, impulse='WIB3M', signif=0.1)
plt.xticks(np.arange(0, 22, 2))
plt.show()

In [ ]:
fred = DataReader(['GDPC1','GDPDEF','FEDFUNDS','M2SL'], 'fred', start='1945', end='2030')

In [ ]:
temp = fred.resample('QS').mean()

iacoviello = 100*np.log(temp).diff(4)
iacoviello['FEDFUNDS'] = temp['FEDFUNDS']
iacoviello = iacoviello.dropna()

iacoviello.columns = ['Y','P','R','M']
iacoviello.head()

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(14, 9))

iacoviello['Y'].to_period('D').plot(ax=ax1)
iacoviello['P'].to_period('D').plot(ax=ax2)
iacoviello['M'].to_period('D').plot(ax=ax3)
iacoviello['R'].to_period('D').plot(ax=ax4)

plt.show()

In [ ]:
model = sm.tsa.VAR(iacoviello[['Y','P','R','M']])
print(model.select_order(8))
model.select_order(8).summary()

In [ ]:
results = model.fit(6)

In [ ]:
results.irf(20).plot()
plt.show()

In [ ]:
results.irf(20).plot(orth=True, impulse='R', signif=0.05)
plt.xticks(np.arange(0, 22, 2))
plt.show()

In [ ]:
results.plot_forecast(12)
plt.show()

In [ ]:
results.fevd(20).plot()
plt.show()